# Loading the required variables

In [1]:
import pandas as pd
from datetime import datetime
import calendar

## Load the dataset into a DataFrame with the specified encoding

In [2]:
# Data is loaded from (https://opendata.transport.nsw.gov.au/dataset/nsw-crash-data)
# Load the dataset from an Excel file (.xlsx format)
# Specify the sheet name if needed, otherwise it defaults to the first sheet
df = pd.read_excel('nsw_road_crash_data_2018-2022_crash.xlsx', sheet_name='Crash Data Table')  
# Display the first few rows to verify the data loaded correctly
print(df.head())

   Crash ID         Degree of crash Degree of crash - detailed  \
0   1151002  Non-casualty (towaway)     Non-casualty (towaway)   
1   1158000                   Fatal                      Fatal   
2   1158001                   Fatal                      Fatal   
3   1158002                   Fatal                      Fatal   
4   1158515                   Fatal                      Fatal   

   Reporting year  Year of crash Month of crash Day of week of crash  \
0            2018           2018       February             Thursday   
1            2018           2018        January               Monday   
2            2018           2018        January               Monday   
3            2018           2018        January               Monday   
4            2018           2018        January              Tuesday   

  Two-hour intervals Street of crash Street type  ...  \
0      06:00 - 07:59       SACKVILLE          ST  ...   
1      00:01 - 01:59         BRUNKER          RD  ...   

### Converting the Year column to integer

In [3]:
df['Year'] = df['Year of crash'].astype(int)

### Filtering the DataFrame to include only the records from 2019 to 2022

In [4]:
df = df[(df['Year'] >= 2019) & (df['Year'] <= 2022)]

### Remove rows where 'time' is 'Unknown'

In [5]:
df = df[df['Two-hour intervals'] != 'Unknown']

### Creating the 'crash_date_time' and 'day' column

In [6]:
# Rename columns
df.rename(columns={
    'Month of crash': 'month',
    'Day of week of crash': 'day',
    'Two-hour intervals': 'time'
}, inplace=True)

# Function to extract center time from 'Two-hour intervals'
def extract_start_time(interval):
    if 'Midnight' in interval:
        return '00:00'  # Midnight as 00:00 in 24-hour format
    else:
        start_time, _ = interval.split(' - ')
        return start_time.strip()  # Return start time without leading or trailing spaces

# Apply function to 'time' column
df['time'] = df['time'].apply(extract_start_time)

### Renaming the 'day of week of crash' to 'day'

In [7]:

df.rename(columns={'Crash ID':'report_id'}, inplace=True)

### Add a new column 'State' with all values set to 'NSW'

In [8]:
df['State'] = 'NSW'

## Creating the 'stats_area' column

In [9]:
def classify_area(conurbation):
    if conurbation == 'Syd-Newc-Woll Gtr conurbation':
        return '1 City'
    elif conurbation == 'Rest of NSW - Urban':
        return '2 Metropolitan'
    elif conurbation in ('Rest of NSW - Rural'):
        return '3 Country'
    else:
        return 'Other'  # In case there are other unaccounted values

# Apply the function to the 'Conurbation 1' column to create a new 'area_type' column
df['stats_area'] = df['Conurbation 1'].apply(classify_area)

### Remove rows where 'stats_area' is 'Other'

In [10]:
df = df[df['stats_area'] != 'Other']

### Creating the 'Loc_type' and 'Location' Column

In [11]:
def classify_location(row):
    if row['Distance'] == 0:
        loc_type = 'Intersection'
        location = f"{row['Street of crash']} {row['Street type']} & {row['Identifying feature']} {row['Identifying feature type']}"
    else:
        loc_type = 'Midblock'
        location = f"{row['Street of crash']} {row['Street type']}"
    return pd.Series([location, loc_type])

# Apply the function to each row and create new columns
df[['Location', 'loc_type']] = df.apply(classify_location, axis=1)

## Creating the 'light_cond' column

In [12]:
# Step 1: Rename the column
df.rename(columns={'Natural lighting': 'light_cond'}, inplace=True)

# Step 2: Replace specific values
df['light_cond'] = df['light_cond'].replace({
    'Darkness': 'Night',
    'Dawn': 'Day',
    'Daylight': 'Day',
    'Dusk': 'Day'
})

# Step 3: Remove rows with 'Unknown'
df = df[df['light_cond'] != 'Unknown']

## Creating the 'weather_cond' column

In [13]:


# Step 1: Rename the column
df.rename(columns={'Weather': 'weather_cond'}, inplace=True)

# Step 2: Replace specific values
df['weather_cond'] = df['weather_cond'].replace({
    'Fine': 'Not Raining',
    'Fog or mist': 'Not Raining',
    'Overcast': 'Not Raining',
    'Other': 'Raining',
    'Raining': 'Raining',
    'Snowing': 'Raining'
})

# Step 3: Remove rows with 'Unknown'
df = df[df['weather_cond'] != 'Unknown']


## Creating the speed limit column

In [14]:
# Filter out 'Unknown' speed limits first
df = df[df['Speed limit'] != 'Unknown']

# Convert 'Speed limit' to integer values after removing ' km/h'
df['Speed limit'] = df['Speed limit'].str.replace(' km/h', '').astype(int)

## Creating the csef_severity column

In [15]:


# Step 1: Rename the column
df.rename(columns={'Degree of crash - detailed': 'csef_severity'}, inplace=True)

# Step 2: Replace specific values
df['csef_severity'] = df['csef_severity'].replace({
    'Fatal': '4: Fatal',
    'Non-casualty (towaway)': '1: PDO',
    'Minor/Other Injury': '2: MI',
    'Moderate Injury': '2: MI',
    'Serious Injury': '3: SI'
})


## Function to standardize column names

In [16]:
def standardize_column_names(columns):
    # Strip whitespace, replace spaces with underscores, and convert to lowercase
    standardized = [col.strip().replace(' ', '_').replace('-', '_').lower() for col in columns]
    return standardized

# Apply the function to the DataFrame's column names
df.columns = standardize_column_names(df.columns)

## Standardizing columns (Keeping only the columns required)

In [17]:
# List of standardized columns to keep
columns_to_keep = [
    'report_id',
    'year',
    'month',
    'day',
    'time',
    'state',
    'stats_area',
    'lga',
    'latitude',
    'longitude',
    'loc_type',
    'location',
    'light_cond',
    'weather_cond',
    'speed_limit',
    'csef_severity'
]

# Select only the specified columns
df = df[columns_to_keep]

## Saving the modified DataFrame to a new CSV file

In [18]:
df.to_csv('Final_NSW.csv', index=False)